Setup and Data Preparation

Define the FL Autoencoder Model (Shared Architecture)

Define the Flower Client (The Core Logic)

 FL Server and Simulation

In [ ]:
# --- CRITICAL: INSTALL LATEST STABLE FLOWER VERSION ---
# This version handles Ray/Python 3.12 compatibility
!pip install -U --force-reinstall "flwr[simulation]" tensorflow scikit-learn numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.0 MB/s eta 0:00:00
  Using cached flwr-1.22.0-py3-none-any.whl.metadata (14 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached cryptography-44.0.3-cp39-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached grpcio_health_checking-1.75.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached iterators-0.0.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached protobuf-4.25.8-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached pycryptodome-3.23.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached ray-2.31.0-cp312-cp312-manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached tomli-2.2.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached tomli_w-1.2.0-py3-none

In [ ]:
# -------------------------------------------------------------------
# STEP 0: SETUP AND IMPORTS
# -------------------------------------------------------------------

# 1. Install necessary dependencies (ensure "flwr[simulation]" is installed)
# NOTE: You must run this command and then click 'RESTART RUNTIME' in Colab
# before running the rest of the code block for Ray to load correctly.
# !pip install -U "flwr[simulation]" tensorflow scikit-learn numpy pandas

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
import flwr as fl
from flwr.common import ndarrays_to_parameters, parameters_to_ndarrays
from sklearn.metrics import roc_auc_score
from collections import Counter

# -------------------------------------------------------------------
# STEP 1: DATA PREPARATION (Final Corrected Cleanup)
# -------------------------------------------------------------------
print("1. Loading and Cleaning Data...")
df_sampled = pd.read_csv("Dataset.csv", low_memory=False)

# A. Separate Features and Target
X = df_sampled.drop('Label', axis=1)
y = df_sampled['Label']

# B. Cleanup: Drop leaky columns, handle NaNs
leaky_cols = [col for col in X.columns if 'Attack Category' in col]
leaky_cols.append('FTP Command Count')
X = X.drop(columns=leaky_cols, errors='ignore')

nan_mask = y.isnull()
X = X[~nan_mask]
y = y[~nan_mask]

for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')
X = X.fillna(0)

# C. Define global variables and CRITICAL type casting
input_dim = X.shape[1]
NUM_CLIENTS = 10

# Define the global test set (CRITICAL: Cast to float32 for Keras/TF)
X_test_global_np = X.to_numpy().astype('float32')
y_test_global_np = y.to_numpy().astype('float32')
print(f"Data ready. Final feature count: {input_dim}")


# -------------------------------------------------------------------
# STEP 2: FL DATA PARTITIONING (Client Splits)
# -------------------------------------------------------------------
print("2. Partitioning Data for FL Clients...")
client_data_splits = []

X_chunks = np.array_split(X, NUM_CLIENTS)
y_chunks = np.array_split(y, NUM_CLIENTS)

for i in range(NUM_CLIENTS):
    X_train_client, X_test_client, y_train_client, y_test_client = train_test_split(
        X_chunks[i], y_chunks[i],
        test_size=0.3, random_state=42, stratify=y_chunks[i]
    )

    # Autoencoder only trains on NORMAL data
    X_train_normal = X_train_client[y_train_client == 0]

    client_data_splits.append({
        # CRITICAL: Cast all client data to float32
        'X_train_normal': X_train_normal.to_numpy().astype('float32'),
        'X_test': X_test_client.to_numpy().astype('float32'),
        'y_test': y_test_client.to_numpy().astype('float32'),
    })
print(f"Data split across {NUM_CLIENTS} simulated clients.")


# -------------------------------------------------------------------
# STEP 3: MODEL DEFINITION (Shared Architecture)
# -------------------------------------------------------------------
def create_autoencoder(input_dim):
    # Encoder (Bottleneck layer)
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    bottleneck = Dense(16, activation='relu', name='bottleneck')(encoded)

    # Decoder (Mirror Image)
    decoded = Dense(64, activation='relu')(bottleneck)
    output_layer = Dense(input_dim, activation='linear')(decoded)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model
print("3. Autoencoder model architecture defined.")


1. Loading and Cleaning Data...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Data ready. Final feature count: 199
2. Partitioning Data for FL Clients...


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represen

Data split across 10 simulated clients.
3. Autoencoder model architecture defined.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
# In Block 2 (Step 4: FL CLIENT LOGIC)

class FederatedAutoencoderClient(fl.client.NumPyClient):
    # ... (init remains the same)

    def get_parameters(self, config):
        # Standard implementation
        return self.model.get_weights()

    def fit(self, parameters, config):
        # Update local model with global parameters (NO manual unwrapping needed)
        self.model.set_weights(fl.common.parameters_to_ndarrays(parameters))

        # Train locally on local NORMAL data (Input = Output)
        X_train = self.data['X_train_normal']

        self.model.fit(
            X_train, X_train,
            epochs=1,     # <-- Keep 1 epoch
            batch_size=16, # <-- CRITICAL: Reduce batch size from 32 to 16
            verbose=0
        )

        # Return updated local weights, size, and metrics dict
        # Standard implementation
        return fl.common.ndarrays_to_parameters(self.model.get_weights()), len(X_train), {}

    # ... (evaluate remains the same)
    def evaluate(self, parameters, config):
        # Evaluation is done by the server for clean comparison
        return 0.0, 0, {}

# FIX for client_fn to handle NumPyClient return compatibility
def client_fn(cid: str) -> fl.client.Client:
    return FederatedAutoencoderClient(int(cid)).to_client() # <--- Explicit conversion FIX

print("4. Federated Client logic redefined with RAW weight handling.")

# -------------------------------------------------------------------
# CRITICAL NOTE: The Server-side needs a minor adjustment too.
# The `initial_parameters` and `fit` method will now work without conversion utilities.
# The server-side evaluation (Step 5) must also be updated.
# -------------------------------------------------------------------

4. Federated Client logic redefined with RAW weight handling.


In [ ]:
from flwr.common import ndarrays_to_parameters
from sklearn.metrics import roc_auc_score
import flwr as fl

# -------------------------------------------------------------------
# 1. Server-Side Global Evaluation Function
# -------------------------------------------------------------------
def evaluate_global_model(server_round: int, parameters, config: dict):
    # This function is now used ONLY for reporting, not for initialization
    model = create_autoencoder(input_dim)

    # Direct use of weights list (NumPy arrays)
    model.set_weights(parameters)

    # Predict reconstruction error on the global test set
    reconstructions = model.predict(X_test_global_np, verbose=0)
    mse = np.mean(np.power(X_test_global_np - reconstructions, 2), axis=1)

    try:
        global_roc_auc = roc_auc_score(y_test_global_np, mse)
    except ValueError:
        global_roc_auc = 0.0

    print(f"Server Round {server_round}: Global ROC-AUC = {global_roc_auc:.4f}")

    # Return 1 - AUC as the loss
    return 1.0 - global_roc_auc, {"roc_auc": global_roc_auc}


# -------------------------------------------------------------------
# 2. Define the Strategy (FedAvg) - MODIFIED
# -------------------------------------------------------------------
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.5,
    fraction_evaluate=0.5,
    min_fit_clients=5,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=create_autoencoder(input_dim).get_weights(),

    # CRITICAL FIX: Set evaluate_fn to None during initialization,
    # and then run evaluation later.
    evaluate_fn=None, # <--- TEMPORARILY DISABLED BUGGY INITIAL EVALUATION

    # Optional: You can re-enable evaluation after Round 1 if needed for paper tracking:
    # on_evaluate_config_fn=lambda rnd: {"round_num": rnd} if rnd > 0 else {}
)

# -------------------------------------------------------------------
# 3. Start the Simulation
# -------------------------------------------------------------------
print("\n5. Starting Federated Learning Simulation (FedAE)...")
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
)

# NOTE: Since we disabled the evaluation in the strategy, the output will only show
# the aggregation rounds. We will analyze the final model from the 'history' object.
print("Federated Learning Simulation Complete.")

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout



5. Starting Federated Learning Simulation (FedAE)...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
2025-10-02 16:20:24,624	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3915222220.0, 'memory': 7830444443.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global

Federated Learning Simulation Complete.


New

In [ ]:
# CRITICAL: Install TFF and Keras
!pip install --quiet tensorflow-federated==0.59.0 tensorflow-cpu==2.15.0 keras==2.15.0
# The above versions are known to be compatible in Colab as of the latest TFF release.


ERROR: Ignored the following versions that require a different python version: 0.34.0 Requires-Python ~=3.9.0; 0.36.0 Requires-Python ~=3.9.0; 0.37.0 Requires-Python >=3.9.0,<3.11; 0.38.0 Requires-Python >=3.9.0,<3.11; 0.39.0 Requires-Python >=3.9.0,<3.11; 0.40.0 Requires-Python >=3.9.0,<3.11; 0.41.0 Requires-Python >=3.9.0,<3.11; 0.42.0 Requires-Python >=3.9.0,<3.11; 0.43.0 Requires-Python >=3.9.0,<3.11; 0.44.0 Requires-Python >=3.9.0,<3.11; 0.45.0 Requires-Python >=3.9.0,<3.11; 0.46.0 Requires-Python >=3.9.0,<3.11; 0.47.0 Requires-Python >=3.9.0,<3.11; 0.48.0 Requires-Python >=3.9.0,<3.11; 0.49.0 Requires-Python >=3.9.0,<3.11; 0.50.0 Requires-Python >=3.9.0,<3.11; 0.51.0 Requires-Python >=3.9.0,<3.11; 0.52.0 Requires-Python >=3.9.0,<3.11; 0.53.0 Requires-Python >=3.9.0,<3.11; 0.54.0 Requires-Python >=3.9.0,<3.11; 0.55.0 Requires-Python >=3.9.0,<3.11; 0.56.0 Requires-Python >=3.9.0,<3.11; 0.57.0 Requires-Python >=3.9.0,<3.11; 0.58.0 Requires-Python >=3.9.0,<3.11; 0.59.0 Requires-Pytho

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from sklearn.metrics import roc_auc_score
from collections import Counter

# Set random seed for reproducibility
tf.random.set_seed(42)

# --- 1. Load Data and Final Cleanup ---
print("1. Loading and Cleaning Data...")
df_sampled = pd.read_csv("Dataset.csv", low_memory=False)

# A. Separate Features and Target
X = df_sampled.drop('Label', axis=1)
y = df_sampled['Label']

# B. Cleanup: Drop leaky columns, handle NaNs
leaky_cols = [col for col in X.columns if 'Attack Category' in col]
leaky_cols.append('FTP Command Count')
X = X.drop(columns=leaky_cols, errors='ignore')

nan_mask = y.isnull()
X = X[~nan_mask]; y = y[~nan_mask]

for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')
X = X.fillna(0)

# C. Define global variables and CRITICAL type casting
input_dim = X.shape[1]
NUM_CLIENTS = 10

# Define the global test set (CRITICAL: Cast to float32 for model consistency)
X_test_global_np = X.to_numpy().astype('float32')
y_test_global_np = y.to_numpy().astype('float32')
print(f"Data ready. Final feature count: {input_dim}")

ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
# -------------------------------------------------------------------
# 2. TFF DATA PARTITIONING AND PREPROCESSING
# -------------------------------------------------------------------
print("2. Partitioning Data for TFF Clients...")
CLIENT_BATCH_SIZE = 32 # Use a small batch size for TFF

def create_client_dataset(X_client):
    """Converts a client's NumPy array into a TFF-compatible dataset."""
    # Autoencoder uses X as both input and target for reconstruction (X, X)
    dataset = tf.data.Dataset.from_tensor_slices((X_client, X_client))
    # Shuffle, repeat, and batch for local training
    return dataset.shuffle(buffer_size=1000).batch(CLIENT_BATCH_SIZE).repeat(1)

client_data_list = []
X_chunks = np.array_split(X, NUM_CLIENTS)
y_chunks = np.array_split(y, NUM_CLIENTS)

for i in range(NUM_CLIENTS):
    # Only train on Normal data for the Autoencoder objective
    X_train_client, _, y_train_client, _ = train_test_split(
        X_chunks[i], y_chunks[i], test_size=0.3, random_state=42, stratify=y_chunks[i]
    )
    X_train_normal = X_train_client[y_train_client == 0].to_numpy().astype('float32')

    client_data_list.append(create_client_dataset(X_train_normal))

# TFF's simulation requires a list of client IDs
client_ids = [f'client_{i}' for i in range(NUM_CLIENTS)]
federated_train_data = tff.simulation.datasets.ClientData.from_clients_and_fn(
    client_ids=client_ids,
    create_tf_dataset_for_client_fn=lambda cid: client_data_list[int(cid.split('_')[1])]
)
print(f"Data partitioned and converted to {len(client_ids)} TFF client datasets.")


# -------------------------------------------------------------------
# 3. TFF MODEL DEFINITION (Using Keras)
# -------------------------------------------------------------------
def create_keras_autoencoder():
    # Encoder
    input_layer = tf.keras.Input(shape=(input_dim,), dtype=tf.float32)
    encoded = tf.keras.layers.Dense(64, activation='relu')(input_layer)
    bottleneck = tf.keras.layers.Dense(16, activation='relu', name='bottleneck')(encoded)
    # Decoder
    decoded = tf.keras.layers.Dense(64, activation='relu')(bottleneck)
    output_layer = tf.keras.layers.Dense(input_dim, activation='linear')(decoded)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

# Define the model function wrapper TFF needs
def model_fn():
    model = create_keras_autoencoder()
    return tff.learning.from_keras_model(
        keras_model=model,
        input_spec=federated_train_data.element_type_structure,
        loss=tf.keras.losses.MeanSquaredError()
    )
print("3. TFF Autoencoder model defined.")

In [ ]:
# -------------------------------------------------------------------
# 4. TFF FEDERATED LEARNING PROCESS
# -------------------------------------------------------------------

# Build the iterative process (TFF's equivalent of a strategy/server)
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01)
)

# Initialize the state of the federated system (Round 0)
state = iterative_process.initialize()
NUM_ROUNDS = 10
print(f"\n4. Starting TFF Federated Simulation for {NUM_ROUNDS} rounds...")

# Run the simulation
history = []
for round_num in range(1, NUM_ROUNDS + 1):
    # Perform one round of federated training
    state, metrics = iterative_process.next(state, [
        federated_train_data.create_tf_dataset_for_client(cid)
        for cid in client_ids
    ])

    # Extract the global model weights
    global_model_weights = tff.learning.models.weights_as_readonly_np(state.model)

    # --- EVALUATION (Manual Reconstruction Error ROC-AUC) ---

    # Load weights into a fresh Keras model for consistent evaluation
    eval_model = create_keras_autoencoder()
    eval_model.set_weights(global_model_weights.trainable)

    # Calculate reconstruction error on the global test set
    reconstructions = eval_model.predict(X_test_global_np, verbose=0)
    mse = np.mean(np.power(X_test_global_np - reconstructions, 2), axis=1)

    # Calculate ROC-AUC score (higher score = better separation)
    roc_auc = roc_auc_score(y_test_global_np, mse)
    history.append((round_num, roc_auc))

    print(f"Round {round_num:2d}: Global ROC-AUC = {roc_auc:.4f}")

print("\nFederated Learning Simulation Complete (TFF).")

In [1]:
import pandas as pd

# Load the final processed dataset
df = pd.read_csv("Dataset.csv", low_memory=False)

# The target column is always 'Label'. We exclude it from the feature count.
feature_columns = df.drop(columns=['Label'], errors='ignore').columns

# Count the number of feature columns
final_feature_count = len(feature_columns)

print(f"Total Columns (including Label): {df.shape[1]}")
print(f"Feature Columns ONLY: {final_feature_count}")

Total Columns (including Label): 215
Feature Columns ONLY: 214
